## Modification: Using network analysis

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('dataset.csv', index_col=0)
id_list = df['user_id'].to_list()

In [40]:
df.head()

,user_id,tweets_count,followers_count,friends_count,has_name,has_image,has_address,has_bio,profile_has_url,present_in_list,friends_to_followers_ratio,label
0,3610511,3057,5470,2385,1,1,1,1,1,1,0.436015,1
1,5656162,3128,506,381,1,1,1,1,1,1,0.752964,1
2,5682702,3158,264,87,1,1,1,1,1,1,0.329545,1
3,6067292,3098,640,622,1,1,0,1,1,1,0.971875,1
4,6015122,2012,62,64,1,1,1,1,1,0,1.032258,1


In [46]:
import collections
import pandas as pd 

# a = [1,1,1,1,2,2,2,2,3,3,4,5,5]

# getting the follower count

df3 = pd.read_csv('followers.csv')
lst = df3['source_id'].to_list()
follower_count = collections.Counter(lst)
followers_count_sorted = sorted(follower_count.items())

# getting friends count
lst2 = df3['target_id'].to_list()
friend_count = collections.Counter(lst2)
friends_count_sorted = sorted(friend_count.items())

user_id_list = list(set(follower_count.keys()) | set(friend_count.keys()) | set(id_list))


In [47]:
df2 = pd.DataFrame({'user_id':sorted(user_id_list)})
df2['total_followers'] = [0] * len(user_id_list)
df2['total_friends'] = [0] * len(user_id_list)
df2.head(15)

,user_id,total_followers,total_friends
0,12,0,0
1,13,0,0
2,14,0,0
3,20,0,0
4,34,0,0
5,87,0,0
6,107,0,0
7,224,0,0
8,246,0,0
9,259,0,0


In [48]:
df_followers = pd.read_csv('followers.csv')

In [50]:
df_friends = pd.read_csv('friends.csv')

### Creating columns to be used for class C attributes

In [59]:
for ind in df_followers.index:
    account = df_followers['source_id'][ind]
    follower = df_followers['target_id'][ind]

    account_follower_count = follower_count[account]
    follower_follower_count = follower_count[follower]

    # df2[df2['user_id'] == account]['total_followers'] += follower_follower_count
    # df2[df2['user_id'] == follower]['total_followers'] += account_follower_count

    account_ind = df2.user_id[df2.user_id == account].index
    follower_ind = df2.user_id[df2.user_id == follower].index

    df2['total_followers'][account_ind] += follower_follower_count
    df2['total_followers'][follower_ind] += account_follower_count

In [60]:
for ind in df_followers.index:
    account = df_followers['source_id'][ind]
    follower = df_followers['target_id'][ind]

    account_friend_count = friend_count[account]
    follower_friend_count = friend_count[follower]

    # df2[df2['user_id'] == account]['total_followers'] += follower_follower_count
    # df2[df2['user_id'] == follower]['total_followers'] += account_follower_count

    account_ind = df2.user_id[df2.user_id == account].index
    follower_ind = df2.user_id[df2.user_id == follower].index

    df2['total_friends'][account_ind] += follower_friend_count
    df2['total_friends'][follower_ind] += account_friend_count

In [62]:
df2.head(30)
df2.to_csv('dataset-total.csv')

### Creating columns for class C attributes

In [14]:
df2['mean_followers'] = [0] * len(id_list)
df2['mean_friends'] = [0] * len(id_list)

In [55]:
for ind in df2.index:
    account = df2['user_id'][ind]
    total_neighbours = 0
    total_neighbours = df[df['user_id'] == account]['followers_count']
    total_neighbours += df[df['user_id'] == account]['friends_count']
    if total_neighbours == 0:
        df2['mean_followers'][ind] = 0
        df2['mean_friends'][ind] = 0 
    else:
        df2['mean_followers'][ind] = df2['total_followers'][ind] / total_neighbours
        df2['mean_friends'][ind] = df2['total_friends'][ind] / total_neighbours

### Append these columns to the dataset

In [18]:
mean_followers_list = df2['mean_followers'].to_list()
mean_friends_list = df2['mean_friends'].to_list()
mean_tweets_list = df2['mean_tweets'].to_list()

df['mean_followers'] = mean_followers_list
df['mean_friends'] = mean_friends_list
df['mean_tweets'] = mean_tweets_list

df.to_csv('dataset-2.csv')